In [9]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
import nltk.data
import collections
import random
from IPython.display import clear_output

In [10]:
# filenames = ["data/reviews_Home_and_Kitchen_5.json"]
# filename = "data/reviews_Home_and_Kitchen_5.json"
file_name = "rawdata_100.pkl"
file_name_s = "rawdata_20.pkl"
filename_adversarial_wordnet = "data_adversial_wordnet"
filename_extension = ".pkl"
filename_adversarial_glove = "data_adversial_glove.pkl"
max_features = 20000
maxlen = 100
batch_size = 10
num_class = 2

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [11]:
POS_TO_WORDNET = {
    'NN': wn.NOUN,
    'JJ': wn.ADJ,
    'JJR': wn.ADJ,
    'JJS': wn.ADJ,
}

In [12]:
def addToReplacement(token, synDict):
    if token[1] not in POS_TO_WORDNET: return None
    w = token[0].lower()
    wn_pos = POS_TO_WORDNET[token[1]]
    synsets = wn.synsets(w, wn_pos)
    if not synsets: return None
    synset = synsets[0]
    synonyms = []
    antonyms = []

    for syn in wn.synsets(w, wn_pos):
        for lem in syn.lemmas():
            if (lem.name() != w):
#                 synonyms.append(lem.name())
                synDict[w] = lem.name()

In [13]:
def makeAdversarialDataset(numReplacement, numSample, texts, data):
    set_adversial = []
    set_unchanged = []
    index = 0
    sampleOutput = []
    for i, row in data.iterrows():
        synsetDictionary = {}

        text = texts[i]
        tokenized = tokenizer.tokenize(text)
        words = word_tokenize(text)
        tokens = nltk.pos_tag(words)
        for token in tokens:
            addToReplacement(token, synsetDictionary)
        if len(synsetDictionary) > 0:
            output = text
            for key in random.sample(synsetDictionary.keys(), min(numReplacement, len(synsetDictionary))):
                output = output.replace(key, synsetDictionary[key])
                if (numSample > 0):
                    sampleOutput.append("[" + str(i) + "]: key-" + str(key) + " value-" + str(synsetDictionary[key]))
                del synsetDictionary[key]
            data.at[i,'reviewText'] = output
            numSample = numSample - 1
            set_adversial.append(i)
        else:
            set_unchanged.append(i)
        print("progress bar: ", index, "/", len(data))
        clear_output(wait=True)
        index = index + 1

    print(sampleOutput)
    return set_adversial

In [ ]:
### Regular dataset

In [6]:
for i in range(3):
    print("Numer of max replacement is ", i + 1, ": ")
    data = pd.read_pickle(file_name)
    texts = data['reviewText']
    set_adversial = makeAdversarialDataset(i + 1, 3, texts, data)
    data = data.loc[set_adversial,:]
    data.to_pickle(filename_adversarial_wordnet + "_" + str(i + 1) + filename_extension)

['[383985]: key-big value-with_child', '[383985]: key-everyday value-daily', '[383985]: key-tile value-roofing_tile', '[204506]: key-great value-with_child', '[204506]: key-happy value-well-chosen', '[204506]: key-door value-threshold', '[4025]: key-chance value-prospect', '[4025]: key-i value-ane', '[4025]: key-terrible value-tremendous']


In [7]:
### replace named entity and numbers]

In [8]:
data = pd.read_pickle(file_name)
texts = data['reviewText']
set_adversial = makeAdversarialDataset(100000, 3, texts, data)
data = data.loc[set_adversial,:]
data.to_pickle(filename_adversarial_wordnet + filename_extension)

['[383985]: key-everyday value-daily', '[383985]: key-big value-with_child', '[383985]: key-dirt value-malicious_gossip', '[383985]: key-tile value-roofing_tile', '[383985]: key-vacuum value-vacuum_cleaner', '[383985]: key-product value-Cartesian_product', '[383985]: key-carpet value-carpeting', '[204506]: key-great value-with_child', '[204506]: key-happy value-well-chosen', '[204506]: key-door value-threshold', '[204506]: key-storage value-warehousing', '[204506]: key-purchase value-leverage', '[204506]: key-product value-Cartesian_product', '[4025]: key-sharp value-precipitous', '[4025]: key-frustration value-foiling', '[4025]: key-bad value-defective', '[4025]: key-chance value-prospect', '[4025]: key-i value-ane', '[4025]: key-terrible value-tremendous']


In [ ]:
### smaller set of 20568 samples

In [15]:
data = pd.read_pickle(file_name_s)
texts = data['reviewText']
set_adversial = makeAdversarialDataset(100000, 3, texts, data)
data = data.loc[set_adversial,:]
data.to_pickle(filename_adversarial_wordnet + "_s" + filename_extension)

['[268965]: key-only value-alone', '[268965]: key-good value-unspoilt', '[268965]: key-night value-Night', '[268965]: key-cup value-loving_cup', '[268965]: key-timer value-timekeeper', '[268965]: key-issue value-publication', '[268965]: key-rest value-quietus', '[268965]: key-make value-shuffling', '[268965]: key-snob value-snoot', '[268965]: key-morning value-dawn', '[268965]: key-stuff value-poppycock', '[268965]: key-maker value-manufacturing_business', '[268965]: key-ground value-undercoat', '[268965]: key-i value-I', '[268965]: key-coffee value-burnt_umber', '[268965]: key-alarm value-alarm_clock', '[268965]: key-husband value-married_man', '[172151]: key-review value-inspection', '[172151]: key-fan value-lover', '[172151]: key-water value-weewee', '[172151]: key-defective value-bad', '[172151]: key-product value-Cartesian_product', '[172151]: key-i value-I', '[172151]: key-resistant value-repellent', '[172151]: key-own value-ain', '[172151]: key-possible value-potential', '[17215

In [17]:
for i in range(3):
    print("Numer of max replacement is ", i + 1, ": ")
    data = pd.read_pickle(file_name_s)
    texts = data['reviewText']
    set_adversial = makeAdversarialDataset(i + 1, 3, texts, data)
    data = data.loc[set_adversial,:]
    data.to_pickle(filename_adversarial_wordnet + "_" + str(i + 1) + "_s" + filename_extension)

['[268965]: key-snob value-snoot', '[268965]: key-issue value-publication', '[268965]: key-good value-unspoilt', '[172151]: key-reason value-grounds', '[172151]: key-resistant value-repellent', '[172151]: key-month value-calendar_month', '[295601]: key-pan value-genus_Pan', '[295601]: key-price value-Mary_Leontyne_Price', '[295601]: key-i value-I']


In [ ]:
### Generate Adversarial samples for training

In [ ]:
data_ori = pd.read_pickle(file_name)

In [ ]:
data = pd.read_pickle(file_name)
texts = data['reviewText']
set_adversial = makeAdversarialDataset(100000, 3, texts, data)
data = data.loc[set_adversial,:]
data.to_pickle(filename_adversarial_wordnet + filename_extension)

In [ ]:
data = pd.read_pickle(file_name)

In [76]:
data.head(10)

,overall,reviewText
489257,1,I use this set for both general food storage i...
318868,0,Not a huge fan . I like to have a side for dam...
527578,1,I have always wanted to have a French press at...
102291,0,I purchased this to replace a breadmaker that ...
137178,0,I bought this pan because my 12 '' pan needed ...
233040,0,"Just sayin ' . I mean , honestly . Do we reall..."
413128,1,"This rack fits the sink perfectly , and is n't..."
149988,0,I bought one of the 20X72 inch mats for my hus...
18029,1,I have four different sizes of these scoops an...
129849,1,These replacement rubbers are perfect for use ...


In [77]:
data = pd.read_pickle(adversarial_file_name)

In [78]:
data.head(10)

,overall,reviewText
489257,1,I use this set for both general food storage i...
318868,0,Not a huge fan . I like to have a side for dam...
527578,1,I have always wanted to have a French press at...
102291,0,I purchased this to replace a breadmaker that ...
137178,0,I bought this pan because my 12 '' pan needed ...
233040,0,"Just sayin ' . I mean , honestly . Do we reall..."
413128,1,"This rack fits the sink perfectly , and is n't..."
149988,0,I bought one of the 20X72 inch mats for my hus...
18029,1,I have four different sizes of these scoops an...
129849,1,These replacement rubbers are perfect for use ...


In [72]:
wn.synsets('dog') # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [73]:
wn.synsets('dog', pos=wn.VERB)

[Synset('chase.v.01')]

In [74]:
wn.synset('dog.n.01')

Synset('dog.n.01')

In [75]:
print(wn.synset('dog.n.01').definition())

a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds


In [76]:
len(wn.synset('dog.n.01').examples())

1

In [77]:
print(wn.synset('dog.n.01').examples()[0])

the dog barked all night


In [78]:
wn.synset('dog.n.01').lemmas()

[Lemma('dog.n.01.dog'),
 Lemma('dog.n.01.domestic_dog'),
 Lemma('dog.n.01.Canis_familiaris')]

In [79]:
[str(lemma.name()) for lemma in wn.synset('dog.n.01').lemmas()]

['dog', 'domestic_dog', 'Canis_familiaris']

In [80]:
wn.lemma('dog.n.01.dog').synset()

Synset('dog.n.01')

In [85]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vwslz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [ ]:
data.to_pickle(adversarial_file_name)

In [ ]:
# def alter_wordnet_antonyms(token):
#     if token[1] not in POS_TO_WORDNET: return None
#     w = token[0].lower()
#     wn_pos = POS_TO_WORDNET[token[1]]
#     synsets = wn.synsets(w, wn_pos)
#     if not synsets: return None
#     synset = synsets[0]
#     synonyms = []
#     antonyms = []

#     for syn in wn.synsets(w, wn_pos):
#         for lem in syn.lemmas():
#             if (lem.name() != w):
#                 synonyms.append(lem.name())
# #     for lem in synset.lemmas():
# #         if lem.antonyms():
# # #                 print("nouns: ", i)
# #             for a in lem.antonyms():
# #                 new_word = a.name()
# #                 if '_' in a.name(): continue
# #                 antonyms.append(new_word)
# #         else:
# #             print("adj: ", i)
# #         synonyms.append(lem.name())
# #     if token[1] == 'NN':
# #         return antonyms
# #     else:
#     return synonyms